# (WIP) Bond Pricing And Accrual Calculation

This notebook will run through the following business use cases :
* Pricing a Bond using the built-in discounting LUSID model fed with a user supplied OIS yield curve.
* Calculating the accrued interest between coupon dates based on user defined Bond instrument parameters such as the day count convention.
* Overriding the calculated accrued interest with user provided value and feeding it in to our Bond valuation.
* <REMOVE - awaiting change. right now feeding in face value makes no real sense> Retrieving Bond PV that has been provided to LUSID (e.g. from a market price source). <>

In doing so we'll cover the following LUSID concepts :
* Defining a LUSID internal representation of a Bond instrument based on user provided parameters.
* Using the StructureMarketData store to hold your OIS yield curve data in way that enables it to be discovered during the Bond valuation process.
* Configuring recipes to run built in LUSID Bond valuation models that make use of the structured data (OIS Yield Curve) you provided.
* Using aggregation requests to return the accrued interest as well as the PV of the Bond based on our instrument definition.
* Using the StructuredResultData store to override the accrued interest calculation and instead use supplied static values.
* Updating our recipes to make use of the StructuredResultData entries in your valuations as oppose to calculating.
* <REMOVE - as above >Upserting Bond market prices as quotes and configuring recipes that value bonds by simply using the quotes.

For this notebook example we'll work with an example Gilt 1.5% 47s:
* Coupon Rate : 1.5%
* Maturity Date: 22 Jul 2047
* Issue Date: 21 Sep 2016
* Coupon Dates : 22 Jan, 22 Jun
* Face Value : £75,000,000

## Setup LUSID

In [42]:
import os
from datetime import datetime, timedelta

import lusid
import pandas as pd
import pytz
from lusid import models
from lusid.utilities import ApiClientFactory
from lusidjam import RefreshingToken


# Authenticate our user and create our API client
from lusidtools.cocoon import load_from_data_frame
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

print ('LUSID Environment Initialised')
print ('LUSID SDK Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

# setup the apis we'll use in this notebook:
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
structured_market_data_api = api_factory.build(lusid.api.StructuredMarketDataApi)
structured_result_data_api = api_factory.build(lusid.api.StructuredResultDataApi)

LUSID Environment Initialised
LUSID SDK Version:  0.6.4688.0


## Test Data Setup

Before we get started let's first setup our test environment data. We'll setup a specific scope for this notebook only
work with a simple transaction portfolio in that scope.

In [9]:
scope = "test-bond-pricing-nb"
portfolio = "single-bond-portfolio-01"

def create_portfolio(scope, portfolio_code, portfolio_name, portfolio_ccy):
    pfs = [[portfolio_code, portfolio_name, portfolio_ccy]]
    pf_df = pd.DataFrame(pfs, columns=['portfolio_code', 'portfolio_name', 'base_currency'])

    portfolio_mapping = {
        "required": {
            "code": "portfolio_code",
            "display_name": "portfolio_name",
            "base_currency": "base_currency",
        },
        "optional": {"created": "$2020-01-01T00:00:00+00:00"},
    }
    result = load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=pf_df,
        mapping_required=portfolio_mapping["required"],
        mapping_optional=portfolio_mapping["optional"],
        file_type="portfolios",
        sub_holding_keys=[],
    )
    print(result)

create_portfolio(scope, portfolio, portfolio, "GBP")

{'portfolios': {'errors': [], 'success': [{'base_currency': 'GBP',
 'created': datetime.datetime(2020, 1, 1, 0, 0, tzinfo=tzutc()),
 'description': None,
 'display_name': 'single-bond-portfolio-01',
 'href': 'https://khalid-local-dev.lusid.com/api/api/portfolios/test-bond-pricing-nb/single-bond-portfolio-01?effectiveAt=2020-06-26T13%3A11%3A26.6120950%2B00%3A00&asAt=2020-06-26T13%3A11%3A26.2664920%2B00%3A00',
 'id': {'code': 'single-bond-portfolio-01', 'scope': 'test-bond-pricing-nb'},
 'is_derived': False,
 'links': [{'description': None,
            'href': 'https://khalid-local-dev.lusid.com/api/api/portfolios/test-bond-pricing-nb/single-bond-portfolio-01/properties?effectiveAt=2020-06-26T13%3A11%3A26.6120950%2B00%3A00&asAt=2020-06-26T13%3A11%3A26.2664920%2B00%3A00',
            'method': 'GET',
            'relation': 'Properties'},
           {'description': None,
            'href': 'https://khalid-local-dev.lusid.com/api/api/portfolios/test-bond-pricing-nb/single-bond-portfolio-0

## Define our Gilt as a Bond Instrument in LUSID

Let's start by defining our Bond instrument in LUSID via the BondInstrument class in the models of the SDKs. Take a look
in the models package for other instruments currently supported (or see the [BondInstrument Specification](https://www.lusid.com/api/swagger/index.html#model-BondInstrument)).

Start by initialising the basic bond parameters for our Gilt:

In [11]:
coupon_rate = 0.015
start_date = datetime(2016, 9, 21, tzinfo=pytz.utc)
maturity_date = datetime(2047, 7, 22, tzinfo=pytz.utc)
dom_ccy = "GBP"
face_value = 75000000

# we will experiment with effective_at data further in notebook when working with accruals between coupon payments.
trade_date = datetime(2020, 6, 22, tzinfo=pytz.utc)
effective_at = datetime(2020, 6, 23, tzinfo=pytz.utc)

Let's now move onto describing the conventions our bond instrument follows. Specifically we'll set the behaviour of
our cash flows date schedule which includes setting the day count convention for calculating accrued interest and
handling cash flows landing on non business days.

All this is behaviour is described  in a FlowConventions object:

In [12]:
gilt_flow_conventions = models.FlowConventions(
        # coupon payment currency
        currency="GBP",
        # semi-annual coupon payments
        payment_frequency=models.Tenor(value=6, unit="M"),
        # using an Actual/365 day count convention (other options : Act360, ActAct, ...
        day_count_convention="Act365",
        # modified following rolling convention (other options : ModifiedPrevious, NoAdjustment, EndOfMonth,...)
        roll_convention="ModifiedFollowing",
        # no holiday calendar supplied
        holiday_calendars=[]
    )

For details on supported tenors, day count and roll conventions see the [BondInstrument Specification](https://www.lusid.com/api/swagger/index.html#model-BondInstrument).

We now have all we need to describe our Gilt as a LUSID instrument:

In [13]:
def create_bond_instrument_definition(start_date, maturity_date, dom_ccy, coupon_rate, face_value, flow_conventions):
    instrument = models.BondInstrument(
        start_date=start_date.isoformat(),
        maturity_date=maturity_date.isoformat(),
        dom_ccy=dom_ccy,
        coupon_rate=coupon_rate,
        principal=face_value,
        flow_conventions=flow_conventions,
        instrument_type="Bond"
    )
    return instrument

bond_instrument_definition = create_bond_instrument_definition(start_date, maturity_date, dom_ccy, coupon_rate, face_value, gilt_flow_conventions)
print(bond_instrument_definition)

{'coupon_rate': 0.015,
 'dom_ccy': 'GBP',
 'flow_conventions': {'currency': 'GBP',
                      'day_count_convention': 'Act365',
                      'holiday_calendars': [],
                      'payment_frequency': {'unit': 'M', 'value': 6},
                      'roll_convention': 'ModifiedFollowing'},
 'identifiers': None,
 'instrument_type': 'Bond',
 'maturity_date': '2047-07-22T00:00:00+00:00',
 'principal': 75000000,
 'start_date': '2016-09-21T00:00:00+00:00'}


Let's use our Bond instrument definition to create an instance of our Gilt 1.5% 47 and upsert it as an instrument into LUSID.

In [15]:
def upsert_bond_instrument(instrument_id, instrument_name, instrument_definition):
    bond_instrument_request = {instrument_id: models.LusidInstrumentDefinition(
        name=instrument_name,
        identifiers={"ClientInternal": models.InstrumentIdValue(instrument_id)},
        definition=instrument_definition
    )}
    # Using upsert_lusid_instrument and not upset_instrument as we're creating an instrument based
    # on a user defined instrument definition.
    response = instruments_api.upsert_lusid_instruments(bond_instrument_request)
    print(response)
    return response

response = upsert_bond_instrument("gilt2047s", "gilt 1.5% 47s", bond_instrument_definition)

# retrieve the instrument id of our upserted gilt to be used later in the notebook
gilt_2047_LUID = response.values['gilt2047s'].lusid_instrument_id

{'failed': {},
 'href': None,
 'links': [{'description': None,
            'href': 'https://khalid-local-dev.lusid.com/api/api/schemas/entities/UpsertInstrumentsResponse',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://khalid-local-dev.lusid.com/app/insights/logs/0HM0POAKOJLLH:00000002',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'values': {'gilt2047s': {'href': 'https://khalid-local-dev.lusid.com/api/api/instruments/LusidInstrumentId/LUID_I1RHNWII',
                          'identifiers': {'ClientInternal': 'gilt2047s',
                                          'LusidInstrumentId': 'LUID_I1RHNWII'},
                          'instrument_definition': {'content': '{\n'
                                                               '  "startDate": '
                        

Before moving on to valuing our bond let's first open a position in the Gilt by supplying LUSID with a transaction for
a single bond assuming the price was at Par:

In [16]:
def upsert_buy_transaction(scope, txn_id, instrument_id, trade_date, effective_at, portfolio):
    gilt_transaction_request = models.TransactionRequest(
        transaction_id=txn_id,
        type="Buy",
        instrument_identifiers={"Instrument/default/ClientInternal": instrument_id},
        transaction_date=trade_date.isoformat(),
        settlement_date=effective_at.isoformat(),
        units=1,
        transaction_price=models.TransactionPrice(price=75000000, type="Price"),
        total_consideration=models.CurrencyAndAmount(amount=75000000, currency="GBP"),
        exchange_rate=1,
        transaction_currency="GBP"
    )

    response = api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_transactions(scope=scope,
                                                                                         code=portfolio,
                                                                                         transaction_request=[gilt_transaction_request])
    print(response)

upsert_buy_transaction(scope, "GiltTXN001", "gilt2047s", trade_date, effective_at, portfolio)

{'href': 'https://khalid-local-dev.lusid.com/api/api/transactionportfolios/test-bond-pricing-nb/single-bond-portfolio-01/transactions?asAt=2020-06-26T13%3A17%3A56.8126900%2B00%3A00',
 'links': [{'description': None,
            'href': 'https://khalid-local-dev.lusid.com/api/api/portfolios/test-bond-pricing-nb/single-bond-portfolio-01?effectiveAt=2020-06-22T00%3A00%3A00.0000000%2B00%3A00&asAt=2020-06-26T13%3A17%3A56.8126900%2B00%3A00',
            'method': 'GET',
            'relation': 'Root'},
           {'description': None,
            'href': 'https://khalid-local-dev.lusid.com/api/api/schemas/entities/UpsertPortfolioTransactionsResponse',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://khalid-local-dev.lusid.com/app/insights/logs/0HM0POAKEHJVC:00000001',
            'method': 'GET',
           

## Defining the Bond valuation

Now that we have our Bond instrument defined and upserted into LUSID we can move onto preparing to execute aggregations
in LUSID to value our Bond holdings.

Aggregations are configured in LUSID through the use of [Recipes]("https://support.finbourne.com/what-is-a-lusid-recipe-and-how-is-it-used") which allow you to configure behaviour. Examples of the type
of configurations include sources market data for specific asset classes, which pricing models to use and where to locate static values that may
be used in calculations.

We'll start with a recipe that simply informs the aggregation engine of which model to use price our Bond. In this notebook
we'll use LUSID's built-in "Discounting" model that prices our bond using an OIS yield curve. But before we run our valuation
we need to supply our yield curve.


### Structured Market Data

[Structured market data]("https://support.finbourne.com/how-do-i-store-and-tr") expands on the simple uploading of market
quotes by allowing you to supply more complex market data into LUSID in a format it can understand.

In the case of our gilt we would like to price it using an OIS yield curve. The source file is in the 'data/GBPOIS50.json' and
consists of a set of discount factors across the maturities of the OIS term structure. While we've hardcoded the rates
in this example LUSID also supports supplying instrument ids or RICs.

<Speak Rz> As well as providing discount factors LUSID also supports forward rate agreements and Swap factors

Let's now load in the yield curves:

In [17]:
# TODO expand as to why this different from other scope
marketDataScope = "FinbourneMarketData"
#
marketSupplier = 'Lusid'

def upsert_ois_yield_curve(ois_curve_json, scope, effective_at, market_asset, ccy):

    ## TODO clean up and elaborate as this currently too messy
    ## a lot going on here we need to describe

    # create request
    structuredDoc = models.StructuredMarketData(document_format="Json", version="1.0.0",
                                                name="DFEUROISCurve", document=ois_curve_json)
    structuredId = models.StructuredMarketDataId(provider="Lusid",price_source=None,
                                                 lineage="MyDemoData", effective_at=effective_at,
                                                 market_element_type="ZeroCurve",
                                                 market_asset=market_asset)
    smdRequest = models.UpsertStructuredMarketDataRequest(market_data_id=structuredId,
                                                          market_data=structuredDoc)

    response = structured_market_data_api.upsert_structured_market_data(
        scope=scope,
        request_body={ccy : smdRequest}
    )
    print(response)

def load_ois_curve_json():
    with open('data/GBPOIS50.json', "r") as myfile:
        return myfile.read()

ois_curve_json = load_ois_curve_json()
upsert_ois_yield_curve(ois_curve_json, marketDataScope, effective_at, "GBP/GBPOIS", "GBP")
upsert_ois_yield_curve(ois_curve_json, marketDataScope, effective_at, "GBP/6M", "GBP")

{'failed': {},
 'href': None,
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://khalid-local-dev.lusid.com/app/insights/logs/0HM0POASAC3DH:00000001',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'values': {'GBP': datetime.datetime(2020, 6, 26, 13, 18, 55, 731562, tzinfo=tzutc())}}
{'failed': {},
 'href': None,
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://khalid-local-dev.lusid.com/app/insights/logs/0HM0POAKEHJVC:00000003',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'values': {'GBP': datetime.datetime(2020, 6, 26, 13, 18, 56, 243931, tzinfo=tzutc())}}


### What Bond Pricing Model To Use?
With our yield curve loaded in and our holding in the bond ready. We're almost ready to run our valuation. We just need to
give LUSID a couple of key instructions to be able to value our bond holding - What Bond pricing model to use? And where
should the aggregatoin engine source it's market data to execute the model? To do so we'll create a simple recipe that
will instruct the LUSID aggregation engine to use the in built LUSID "Discounting" model.

The PricingContext is used to select the pricing model as well offering the options of additional parameters that configure the
model behaviour. See the [Swagger spec]("https://www.lusid.com/api/swagger/index.html#model-PricingContext") for more details.

In [18]:
#TODO explain more of these arguments

def create_discounting_bond_pricing_context():
    return models.PricingContext(
        options=models.PricingOptions(
            allow_any_instruments_with_sec_uid_to_price_off_lookup=True,
            produce_separate_result_for_linear_otc_legs=False,
            allow_partially_successful_evaluation=True
        ),
        model_rules=[
            models.VendorModelRule(
                supplier="Lusid",
                model_name="Discounting",
                instrument_type="Bond",
                parameters="{}"
            )
        ]
    )

pricing_context = create_discounting_bond_pricing_context()

### What Market Data Should the Model Use?

The MarketContext is how we inform LUSID where to retrieve market data for a given aggregatoin. In our case we need to
tell LUSID where our OIS yield curve was stored (the market_data_scope and supplier). See the [Swagger spec]("https://www.lusid.com/api/swagger/index.html#model-MarketContext")
for more details.

In [30]:
def create_market_context():
    return models.MarketContext(
        market_rules=[
            models.MarketDataKeyRule(
                key="Rates.*.*",
                data_scope=marketDataScope,
                supplier=marketSupplier,
                quote_type='Rate',
                field='Mid')
        ],
        options=models.MarketOptions(
            default_supplier=marketSupplier,
            default_scope=marketDataScope,
            # TODO describ manifest or further down??
            manifest_level_of_detail="Full")
    )

    return market_context

market_context = create_market_context()

And now we can create our recipe:

In [31]:
def upsert_discount_bond_pricing_recipe(scope, recipe_name, market_context, pricing_context):

    cfgRecipe = models.ConfigurationRecipe(
        scope=scope,
        code=recipe_name,
        description="Price bond using discounting model",
        market=market_context,
        pricing=pricing_context
    )


    #upsert the recipe
    response = configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=models.UpsertRecipeRequest(configuration_recipe=cfgRecipe)
    )

    print(response)

discounting_bond_recipe = "test-discounting-bond"
upsert_discount_bond_pricing_recipe(scope, discounting_bond_recipe, market_context, pricing_context)

{'href': None,
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://khalid-local-dev.lusid.com/app/insights/logs/0HM0POAOE7T8P:00000001',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'value': datetime.datetime(2020, 6, 26, 13, 42, 35, 561560, tzinfo=tzutc())}


## Valuing our Bond Holdings

Let's quickly summarise our current state:
 * We've defined a Gilt 1.5% 47s bond (including defining it's date conventions)
 * We've loaded our OIS yield curve into the Structure Market Data store.
 * We've setup our Recipe configuring how we would like to value our bond.
 * We have a holding of 1 unit of the Gilt in our portfolio.

We're now ready to run our aggregation and value our Bond holdings. We simply need to inform LUSID what values we
would like to aggregate. In our case our entire portfolio is made up of one unit of the Gilt so we simply require
the PV of the holding.

In [34]:
def run_bond_pricing_aggregation(scope, recipe, effective_at):
    aggregation_request = models.AggregationRequest(
        effective_at=effective_at.isoformat(),
        recipe_id=models.ResourceId(scope=scope, code=recipe),
        metrics=[
            models.AggregateSpec(key='Holding/default/PV', op='Value'),
        ]
    )

    return api_factory.build(lusid.api.AggregationApi).get_aggregation(scope=scope, code=portfolio,
                                                             aggregation_request=aggregation_request)



aggregation_result = run_bond_pricing_aggregation(scope, discounting_bond_recipe, effective_at)
bond_holdings = aggregation_result.data[0]['Holding/default/PV']
print(f"Bonds Holdings (as at {effective_at}) : {bond_holdings}")


Bonds Holdings (as at 2020-06-23 00:00:00+00:00) : 103998646.71475208


We've now priced our Gilt holdings using the LUSID internal Bond "Discounting" model. As our recipe is already setup
we can very seamlessly revalue our bond holdings for the following day:

In [37]:
effective_at_plus_one = effective_at + timedelta(days=1)
aggregation_result = run_bond_pricing_aggregation(scope, discounting_bond_recipe, effective_at_plus_one)
bond_holdings = aggregation_result.data[0]['Holding/default/PV']
print(f"Bonds Holdings (as at {effective_at_plus_one}) : {bond_holdings}")

Bonds Holdings (as at 2020-06-23 00:00:00+00:00) : 104001728.9065329


## Accrued Interest

The above example shows our Bond holding changed from one day to the next. This can be explained by the accrued
interest on the Gilt. Recall that when we defined our gilt we set FlowConventions that contain all the information needed to calculate
accrued interest between coupon dates. The 'Holding/default/PV' aggregation generates a clean price using the "Discounting" model before
calculating and adding accrued interest.


To display the accrued interest we simply add the request the next time we run an aggregation:

In [41]:
def run_bond_pricing_aggregation_and_accrued(scope, recipe, effective_at, ):
    aggregation_request = models.AggregationRequest(
        effective_at=effective_at.isoformat(),
        recipe_id=models.ResourceId(scope=scope, code=recipe),
        metrics=[
            models.AggregateSpec(key='Holding/default/PV', op='Value'),
            models.AggregateSpec(key='Holding/default/Accrual', op='Value')
        ]
    )

    return api_factory.build(lusid.api.AggregationApi).get_aggregation(scope=scope, code=portfolio,
                                                             aggregation_request=aggregation_request)

aggregation_result = run_bond_pricing_aggregation_and_accrued(scope, discounting_bond_recipe, effective_at)
bond_holdings = aggregation_result.data[0]['Holding/default/PV']
accrued_interest = aggregation_result.data[0]['Holding/default/Accrual']
print(f"Bonds Holdings (as at {effective_at}) : {bond_holdings}")
print(f"Accrued Interest (as at {effective_at}) : {accrued_interest}")

aggregation_result = run_bond_pricing_aggregation_and_accrued(scope, discounting_bond_recipe, effective_at_plus_one)
bond_holdings_t_1 = aggregation_result.data[0]['Holding/default/PV']
accrued_interest_t_1 = aggregation_result.data[0]['Holding/default/Accrual']
print(f"Bonds Holdings (as at {effective_at_plus_one}) : p{bond_holdings_t_1} (dtd: {bond_holdings_t_1-bond_holdings})")
print(f"Accrued Interest (as at {effective_at_plus_one}) : {accrued_interest_t_1} (dtd: {accrued_interest_t_1-accrued_interest})")

Bonds Holdings (as at 2020-06-23 00:00:00+00:00) : 103998646.71475208
Accrued Interest (as at 2020-06-23 00:00:00+00:00) : 471575.34246575343
Bonds Holdings (as at 2020-06-24 00:00:00+00:00) : 104001728.9065329 (dtd: 3082.191780820489)
Accrued Interest (as at 2020-06-24 00:00:00+00:00) : 474657.5342465753 (dtd: change: 3082.191780821886)


## Accrual Override

Up until now we have relied on LUSID to run it's internal bond pricing model as well as calculate accruals based on
conventions defined in the Gilt instrument. However you may require to use a different value for accrued interest - one
that has been calculated externally for example. To do so we need to address two concerns. How do we load a one off accrual
into LUSID? And how do we ensure LUSID uses that accrual during aggregation as oppose to reverting to a calculating it
on the fly as in the previous examples.

### Structured Result Store

The [Structured Result Store]("https://support.finbourne.com/how-do-i-store-and-retrieve-structured-market-data-documents") is
a location to store non quote data that may nevertheless be used in an aggregation. Examples include YTD performance on
an index, sensitivties of a swap or in our case the accrued interest.

Just as with the structured market data store when results need to ensure they can be retrieved by LUSID during
an aggregation which is why we must specific a key that uniquely identifies the data

In [ ]:
result_data_scope = "Finbourne-Examples"

def get_accrual_csv_output(instrument_id, accrual):
    return "LusidInstrumentId,Accrual" + "\r\n" + f"{instrument_id},0.069109979"


def upsert_structured_result_data_overrides(effective_at,instrument_id):
    accrual_document = get_accrual_csv_output(instrument_id, '0.069109979')

    accrual_result = models.StructuredResultData(
        # TODO change to JSON
        document_format="CSV",
        version="1.0.0",
        name="IRS accrual",
        document=accrual_document
    )

    # create identifier
    accrual_result_id = models.StructuredResultDataId(
        source="Client",
        code="GiltAccrual",
        effective_at=effective_at.isoformat(),
        result_type="UnitResult/Analytic"
    )

    # create structured request
    structured_request = models.UpsertStructuredResultDataRequest(
        id=accrual_result_id,
        data=accrual_result
    )

    # upsert
    response = structured_result_data_api.upsert_structured_result_data(
        scope=result_data_scope,
        request_body={"AccrualOR1": structured_request}
    )

    print(response)

Now that the accrual is in LUSID we need to address our second concern of notifying the aggregation to override the
accrual calculation with our value when required. We can achieve this via the use of a Result Data Rules. These rules
rely on a pattern that when matched signals to the aggregation that the stored result data should be used over any
calculated value.

In [ ]:
def create_accrual_result_data_rule(result_data_scope):
    # TODO exaplin this key in more detail not making much sense why it maps to accrual specifically
    accrual_key_rule = models.ResultDataKeyRule(
        resource_key="UnitResult/Isin/Yield",
        supplier="Client",
        data_scope=result_data_scope,
        document_code="GiltAccrual"
    )

    return accrual_key_rule


create_accrual_result_data_rule(result_data_scope)

We know need to let LUSID know to apply this rule. To do so we simply add our rule to a pricing context definition which
you call informs LUSID on what models to use for the aggregation.

In [ ]:
def create_discounting_bond_pricing_context_with_accrual_override(accrual_key_rule):
    return models.PricingContext(
        options=models.PricingOptions(
            allow_any_instruments_with_sec_uid_to_price_off_lookup=True,
            produce_separate_result_for_linear_otc_legs=False,
            allow_partially_successful_evaluation=True
        ),
        model_rules=[
            models.VendorModelRule(
                supplier="Lusid",
                model_name="Discounting",
                instrument_type="Bond",
                parameters="{}"
            )
        ],
        result_data_rules=[accrual_key_rule]
    )

create_discounting_bond_pricing_context_with_accrual_override(accrual_key_rule)

In [ ]:
# def create_accrued_interest_config_recipe():
#     cfgRecipe = models.ConfigurationRecipe(
#         #scope="Finbourne-Examples",# added by KK -> onl on updates sdk
#         code="AccruedInterestOnly",
#         description="Calculate the accrued interest"
#     )
#
#
#     #upsert the recipe
#     response = api_factory.build(lusid.api.ConfigurationRecipeApi).upsert_configuration_recipe(
#         scope="Finbourne-Examples",
#         structured_data=models.UpsertRecipeRequest(code=recipe_code,
#                                                    configuration_recipe=cfgRecipe)
#     )